# Module 3: Embeddings Deep Dive

**Level:** Intermediate  
**Prerequisites:** Modules 1 & 2 completed

---

## Learning Objectives

By the end of this module, you will be able to:

- Understand what embeddings are and how they work
- Generate embeddings using different models
- Calculate similarity between embeddings
- Choose the right embedding model for your use case
- Implement semantic search with embeddings
- Understand embedding dimensions and trade-offs

---

# 1. What Are Embeddings?

## 1.1 The Core Concept

**Simple Definition:**
> Embeddings are numerical representations of text that capture meaning.

**Think of it like this:**
- Words are converted to numbers (vectors)
- Similar meanings → Similar numbers
- Computers can then do math with meaning!

**Example:**
```
"cat"     → [0.2, -0.5, 0.8, 0.1, ...] (384 numbers)
"kitten"  → [0.3, -0.4, 0.7, 0.2, ...] (similar numbers!)
"car"     → [-0.1, 0.6, -0.3, 0.9, ...] (very different numbers)
```

## 1.2 Why Do We Need Embeddings?

**Problem:** Computers don't understand text

**Traditional Approach (doesn't work for RAG):**
- Keyword matching: "cat" only matches "cat"
- Misses "kitten", "feline", "kitty"
- Can't understand meaning or context

**Embeddings Approach (what RAG uses):**
- Captures semantic meaning
- "cat" is similar to "kitten", "feline", "pet"
- Finds relevant content even with different words

**This is why embeddings enable semantic search!**

---

# 2. Generating Your First Embeddings

## 2.1 Setup

We'll use **sentence-transformers** - a popular, free, open-source library.

In [ ]:
# Install required library
!pip install -q sentence-transformers

In [28]:
from sentence_transformers import SentenceTransformer
import numpy as np

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


## 2.2 Load an Embedding Model

In [29]:
# Load a small, fast embedding model
print("Loading embedding model...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Model loaded!")
print(f"Model produces {model.get_sentence_embedding_dimension()} dimensional embeddings")

Loading embedding model...
✅ Model loaded!
Model produces 384 dimensional embeddings


## 2.3 Generate Embeddings

In [30]:
# Simple example
text = "The cat sat on the mat"

# Generate embedding
embedding = model.encode(text)

print(f"Original text: {text}")
print(f"Embedding shape: {embedding.shape}")
print(f"Embedding type: {type(embedding)}")
print(f"\nFirst 10 values: {embedding[:10]}")

Original text: The cat sat on the mat
Embedding shape: (384,)
Embedding type: <class 'numpy.ndarray'>

First 10 values: [ 0.13040186 -0.01187012 -0.02811704  0.05123863 -0.05597441  0.03019154
  0.03016129  0.02469839 -0.01837056  0.05876678]


### 💡 What just happened?

- Input: 6 words
- Output: 384 numbers (a vector)
- These numbers capture the meaning of the sentence
- We can now do math with this meaning!

---

# 3. Similarity: The Heart of RAG

## 3.1 Cosine Similarity Explained

**Cosine similarity** measures how similar two vectors are.

**Range:** -1 to 1
- **1.0** = Identical meaning
- **0.8-0.9** = Very similar
- **0.5-0.7** = Somewhat related
- **< 0.3** = Not related

**Formula:** Don't worry about the math - just use the function!

In [31]:
def cosine_similarity(vec1, vec2):
    """
    Calculate cosine similarity between two vectors.
    
    Returns a score between -1 and 1 (higher = more similar)
    """
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

print("✅ Similarity function ready!")

✅ Similarity function ready!


## 3.2 Testing Similarity

In [32]:
# Create test sentences
sentences = [
    "The cat sat on the mat",
    "A feline rested on the rug",      # Similar meaning, different words
    "Dogs are loyal animals",          # Different topic
    "Python is a programming language" # Completely unrelated
]

# Generate embeddings for all sentences
embeddings = model.encode(sentences)

# Compare first sentence to all others
print("Comparing to: 'The cat sat on the mat'\n")
for i, sentence in enumerate(sentences):
    similarity = cosine_similarity(embeddings[0], embeddings[i])
    print(f"Similarity to '{sentence}'")
    print(f"Score: {similarity:.3f}\n")

Comparing to: 'The cat sat on the mat'

Similarity to 'The cat sat on the mat'
Score: 1.000

Similarity to 'A feline rested on the rug'
Score: 0.564

Similarity to 'Dogs are loyal animals'
Score: 0.165

Similarity to 'Python is a programming language'
Score: 0.031



### 💡 Observations

Notice how:
- ✅ "A feline rested on the rug" has HIGH similarity (same meaning, different words)
- ✅ "Dogs are loyal animals" has MEDIUM similarity (animals, but different context)
- ✅ "Python is a programming language" has LOW similarity (completely different)

**This is semantic search in action!**

---

# 4. Building a Simple Semantic Search

## 4.1 Create a Document Collection

In [35]:
# Sample knowledge base
documents = [
    "Python is a high-level programming language known for simplicity",
    "Machine learning enables computers to learn from data",
    "Neural networks are inspired by biological brains",
    "Dogs are loyal and friendly pets that need exercise",
    "Cats are independent animals that make great companions",
    "JavaScript is used for web development and runs in browsers",
    "Deep learning uses multi-layered neural networks",
    "Puppies require training and socialization from an early age"
]

print(f"Knowledge base: {len(documents)} documents")

Knowledge base: 8 documents


## 4.2 Embed All Documents

In [36]:
# Generate embeddings for all documents
print("Generating embeddings for all documents...")
doc_embeddings = model.encode(documents)

print(f"✅ Created {len(doc_embeddings)} embeddings")
print(f"Each embedding has {doc_embeddings[0].shape[0]} dimensions")

Generating embeddings for all documents...
✅ Created 8 embeddings
Each embedding has 384 dimensions


## 4.3 Search Function

In [ ]:
def search(query, documents, doc_embeddings, top_k=3):
    """
    Search for documents similar to the query.
    
    Args:
        query: Search query (string)
        documents: List of document texts
        doc_embeddings: Pre-computed document embeddings
        top_k: Number of results to return
    
    Returns:
        List of (document, similarity_score) tuples
    """
    # Embed the query
    query_embedding = model.encode(query)
    
    # Calculate similarities
    similarities = []
    for i, doc_emb in enumerate(doc_embeddings):
        similarity = cosine_similarity(query_embedding, doc_emb)
        similarities.append((documents[i], similarity))
    
    # Sort by similarity (highest first)
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Return top k results
    return similarities[:top_k]

print("✅ Search function ready!")

## 4.4 Try It Out!

In [37]:
# Test different queries
queries = [
    "What is artificial intelligence?",
    "Tell me about pet dogs",
    "How do I code in Python?"
]

for query in queries:
    print(f"\n{'='*80}")
    print(f"QUERY: {query}")
    print(f"{'='*80}")
    
    results = search(query, documents, doc_embeddings, top_k=3)
    
    for i, (doc, score) in enumerate(results, 1):
        print(f"\n{i}. (Score: {score:.3f})")
        print(f"   {doc}")


QUERY: What is artificial intelligence?

1. (Score: 0.408)
   Machine learning enables computers to learn from data

2. (Score: 0.395)
   Neural networks are inspired by biological brains

3. (Score: 0.326)
   Python is a high-level programming language known for simplicity

QUERY: Tell me about pet dogs

1. (Score: 0.548)
   Dogs are loyal and friendly pets that need exercise

2. (Score: 0.437)
   Puppies require training and socialization from an early age

3. (Score: 0.413)
   Cats are independent animals that make great companions

QUERY: How do I code in Python?

1. (Score: 0.554)
   Python is a high-level programming language known for simplicity

2. (Score: 0.148)
   Puppies require training and socialization from an early age

3. (Score: 0.138)
   JavaScript is used for web development and runs in browsers


### 🎯 Amazing, right?

Notice how:
- "What is artificial intelligence?" finds ML and neural network docs
- "Tell me about pet dogs" finds dog-related docs
- "How do I code in Python?" finds Python programming doc

**Even though the exact words don't match!** This is the power of embeddings.

---

# 5. Understanding Embedding Models

## 5.1 Popular Embedding Models

### Free / Open Source Models

| Model | Dimensions | Speed | Best For |
|-------|-----------|-------|----------|
| **all-MiniLM-L6-v2** | 384 | ⚡⚡⚡ Fast | General purpose, **recommended starter** |
| **all-mpnet-base-v2** | 768 | ⚡⚡ Medium | Better quality, slower |
| **all-MiniLM-L12-v2** | 384 | ⚡⚡ Medium | Balance of speed and quality |

### API-Based Models (require API key)

| Model | Dimensions | Cost | Best For |
|-------|-----------|------|----------|
| **OpenAI text-embedding-3-small** | 1536 | $ | High quality, general |
| **OpenAI text-embedding-3-large** | 3072 | $$ | Best quality, expensive |
| **Cohere embed-english-v3.0** | 1024 | $ | Good quality, affordable |

### Domain-Specific Models

| Domain | Model | Why |
|--------|-------|-----|
| **Code** | code-search-net | Understands code syntax |
| **Science** | allenai-specter | Trained on scientific papers |
| **Multilingual** | paraphrase-multilingual-mpnet-base-v2 | Supports 50+ languages |

## 5.2 Comparing Different Models

In [38]:
# Load two different models for comparison
print("Loading models...\n")

model_small = SentenceTransformer('all-MiniLM-L6-v2')      # 384 dimensions
model_large = SentenceTransformer('all-mpnet-base-v2')     # 768 dimensions

print("✅ Both models loaded!")
print(f"Small model: {model_small.get_sentence_embedding_dimension()} dimensions")
print(f"Large model: {model_large.get_sentence_embedding_dimension()} dimensions")

Loading models...

✅ Both models loaded!
Small model: 384 dimensions
Large model: 768 dimensions


In [39]:
# Compare on a similarity task
test_pairs = [
    ("The dog is running", "A canine is jogging"),           # Similar
    ("I love pizza", "Pizza is delicious"),                  # Related
    ("Python programming", "Cooking pasta")                  # Unrelated
]

print("Comparing model performance:\n")
for text1, text2 in test_pairs:
    # Small model
    emb1_small = model_small.encode([text1, text2])
    sim_small = cosine_similarity(emb1_small[0], emb1_small[1])
    
    # Large model  
    emb1_large = model_large.encode([text1, text2])
    sim_large = cosine_similarity(emb1_large[0], emb1_large[1])
    
    print(f"Pair: '{text1}' vs '{text2}'")
    print(f"  Small model: {sim_small:.3f}")
    print(f"  Large model: {sim_large:.3f}")
    print()

Comparing model performance:

Pair: 'The dog is running' vs 'A canine is jogging'
  Small model: 0.818
  Large model: 0.827

Pair: 'I love pizza' vs 'Pizza is delicious'
  Small model: 0.801
  Large model: 0.785

Pair: 'Python programming' vs 'Cooking pasta'
  Small model: 0.142
  Large model: 0.120



### 💡 Model Selection Guide

**Choose based on:**

1. **Speed vs Quality**
   - Need fast? → all-MiniLM-L6-v2
   - Need best quality? → all-mpnet-base-v2 or API models

2. **Cost**
   - Free / Local → sentence-transformers models
   - Budget for API → OpenAI, Cohere

3. **Domain**
   - General text → all-MiniLM-L6-v2 (start here!)
   - Code → code-specific model
   - Science → scientific model
   - Multiple languages → multilingual model

**For most cases: Start with all-MiniLM-L6-v2, then upgrade if needed.**

---

# 6. Embedding Dimensions Explained

## 6.1 What Are Dimensions?

**Dimensions = How many numbers represent each piece of text**

**Common sizes:**
- 384 dimensions (small, fast)
- 768 dimensions (medium)
- 1536 dimensions (large, high quality)

## 6.2 The Trade-off

### More Dimensions
**Pros:**
- Can capture more nuanced meaning
- Better quality (usually)
- Better performance on complex tasks

**Cons:**
- Slower to compute
- More storage space
- More expensive (if using APIs)

### Fewer Dimensions
**Pros:**
- Faster computation
- Less storage
- Lower cost

**Cons:**
- May miss subtle differences
- Lower quality (sometimes)

**Reality:** For most RAG use cases, 384-768 dimensions work great!

---

# 7. Practical Tips for Using Embeddings

## 7.1 Best Practices

### ✅ DO:

1. **Use the same model for queries and documents**
   - MUST use identical model for both
   - Different models = incompatible vectors

2. **Embed at the right granularity**
   - Usually: Embed chunks, not whole documents
   - From Module 2: 300-600 token chunks work well

### ❌ DON'T:

1. **Mix different embedding models**
   - Can't compare vectors from different models
   - Must re-embed everything if changing models

2. **Embed extremely long texts**
   - Models have max length (usually 256-512 tokens)
   - Longer text gets truncated
   - Chunk first, then embed

3. **Re-embed everything on every query**
   - Wasteful and slow
   - Embed documents once, store embeddings

## 7.2 Common Issues and Solutions

### Issue 1: Similarity Scores Too Low
**Problem:** All similarities < 0.3
**Solution:** 
- Check if using same model for query and docs
- Try a better quality model
- Check if chunks are too large/small

### Issue 2: Everything Seems Similar
**Problem:** All similarities > 0.8  
**Solution:**
- Chunks might be too similar (duplicate content)
- Model might not be domain-appropriate
- Need more diverse document set

### Issue 3: Slow Embedding Generation
**Problem:** Takes too long to embed   
**Solution:**
- Use smaller model (384 dims vs 768)
- Batch process documents
- Use GPU if available
- Consider API-based models for scale

---

# 8. Complete Example: Building a Mini RAG Retriever

Let's combine chunking (Module 2) with embeddings (Module 3) to build a simple retriever!

In [ ]:
class SimpleRetriever:
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        """
        Initialize retriever with embedding model.
        """
        self.model = SentenceTransformer(model_name)
        self.chunks = []
        self.embeddings = None
    
    def add_documents(self, documents, chunk_size=500):
        """
        Add documents to the retriever (chunks and embeds them).
        """
        # Simple chunking (from Module 2)
        for doc in documents:
            words = doc.split()
            for i in range(0, len(words), chunk_size):
                chunk = ' '.join(words[i:i+chunk_size])
                self.chunks.append(chunk)
        
        # Generate embeddings
        print(f"Embedding {len(self.chunks)} chunks...")
        self.embeddings = self.model.encode(self.chunks)
        print(f"✅ Ready! {len(self.chunks)} chunks indexed.")
    
    def search(self, query, top_k=3):
        """
        Search for relevant chunks.
        """
        # Embed query
        query_embedding = self.model.encode(query)
        
        # Calculate similarities
        similarities = []
        for i, chunk_emb in enumerate(self.embeddings):
            sim = cosine_similarity(query_embedding, chunk_emb)
            similarities.append((self.chunks[i], sim))
        
        # Sort and return top k
        similarities.sort(key=lambda x: x[1], reverse=True)
        return similarities[:top_k]

print("✅ SimpleRetriever class ready!")

In [ ]:
# Test it with sample documents
sample_docs = [
    """
    Python is a versatile programming language widely used in web development,
    data science, and automation. Its simple syntax makes it beginner-friendly
    while remaining powerful for advanced applications.
    """,
    """
    Machine learning is a subset of artificial intelligence that enables systems
    to learn and improve from experience. Popular frameworks include TensorFlow,
    PyTorch, and scikit-learn.
    """,
    """
    Dogs are loyal companions that require regular exercise, training, and
    veterinary care. Different breeds have varying needs and temperaments.
    """
]

# Create retriever and add documents
retriever = SimpleRetriever()
retriever.add_documents(sample_docs, chunk_size=100)

# Test searches
test_queries = [
    "How do I start learning to code?",
    "What is AI and machine learning?",
    "Tell me about caring for pets"
]

for query in test_queries:
    print(f"\n{'='*80}")
    print(f"Query: {query}")
    print(f"{'='*80}")
    results = retriever.search(query, top_k=2)
    for i, (chunk, score) in enumerate(results, 1):
        print(f"\nResult {i} (Score: {score:.3f}):")
        print(chunk.strip())

### 🎉 You just built a retriever!

This combines:
- ✅ Chunking (Module 2)
- ✅ Embeddings (Module 3)
- ✅ Similarity search

**Next step:** Add vector storage (Module 4) for better performance!

---

# 9. Summary

## Key Takeaways

1. **Embeddings convert text to numbers** that capture semantic meaning

2. **Similar meanings → Similar numbers** (vectors)

3. **Cosine similarity** measures how similar two embeddings are (0-1 scale)

4. **Model choice matters:**
   - Start with: all-MiniLM-L6-v2 (384 dims)
   - Need better quality: all-mpnet-base-v2 (768 dims)
   - Specific domain: Use domain-specific models

5. **Must use same model** for documents and queries

6. **Embed once, store forever** - don't re-embed on every query

7. **Dimensions:** More isn't always better - 384-768 works for most cases

8. **This enables semantic search** - the core of RAG retrieval!

# 🎯 Practice Exercises

## Exercise 1: Understanding Embedding Similarity

### Task
Explore how embeddings capture semantic similarity between different texts.

### Instructions

1. Use the sample sentences below:

```python
sentences = [
    "The dog is playing in the park",
    "A puppy is running outside",
    "The cat is sleeping on the couch",
    "Python is a programming language",
    "Machine learning models need data",
    "I love coding in Python"
]
```

2. Generate embeddings for all sentences using `all-MiniLM-L6-v2`

3. Calculate similarity scores between:
   - Sentence 1 and all others
   - Sentence 4 and all others

4. Answer these questions:
   - Which sentences are most similar to "The dog is playing in the park"?
   - Which sentences are most similar to "Python is a programming language"?
   - What similarity threshold would you use to filter unrelated content?

### Expected Output

```
Similarity Analysis:

Query: "The dog is playing in the park"
Most similar: ?
Least similar: ?
Observations: ?

Query: "Python is a programming language"
Most similar: ?
Least similar: ?
Observations: ?

Recommended similarity threshold: ?
```

## Exercise 2: Chunk Size Impact on Retrieval

### Task
Understand how chunk size affects retrieval quality.

### Instructions

1. Use this long document:

```python
document = """
Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to
the natural intelligence displayed by humans and animals. Leading AI textbooks define
the field as the study of intelligent agents: any device that perceives its environment
and takes actions that maximize its chance of successfully achieving its goals.

Machine learning is a subset of artificial intelligence that focuses on the use of data
and algorithms to imitate the way that humans learn, gradually improving its accuracy.
Machine learning is an important component of the growing field of data science.

Deep learning is part of a broader family of machine learning methods based on artificial
neural networks with representation learning. Learning can be supervised, semi-supervised
or unsupervised. Deep learning architectures such as deep neural networks, deep belief
networks, recurrent neural networks and convolutional neural networks have been applied
to fields including computer vision, speech recognition, natural language processing,
machine translation, and bioinformatics.

Natural language processing is a subfield of linguistics, computer science, and artificial
intelligence concerned with the interactions between computers and human language, in
particular how to program computers to process and analyze large amounts of natural
language data. Challenges in natural language processing frequently involve speech
recognition, natural language understanding, and natural language generation.
"""
```

2. Chunk this document using 3 different sizes:
   - Small: 100 characters
   - Medium: 200 characters
   - Large: 400 characters

3. For each chunk size:
   - Create embeddings
   - Test with query: "What is machine learning?"
   - Retrieve top 3 chunks

4. Compare results:
   - Which chunk size gave the most focused answer?
   - Which gave the most complete answer?
   - Which had the best balance?

### Expected Output

```
Chunk Size Comparison:

Small Chunks (100 chars):
- Number of chunks: ?
- Top result: "?"
- Score: ?
- Analysis: ?

Medium Chunks (200 chars):
- Number of chunks: ?
- Top result: "?"
- Score: ?
- Analysis: ?

Large Chunks (400 chars):
- Number of chunks: ?
- Top result: "?"
- Score: ?
- Analysis: ?

Best chunk size for this use case: ? because ?
```


## What's Next?

In **Module 4: Vector Storage & Retrieval**, you'll learn:
- How to store millions of embeddings efficiently
- Vector databases (FAISS, Chroma, Pinecone)
- Fast similarity search at scale
- Building a production-ready retriever

Get ready to handle real-world scale! 🚀